# Ejemplo de Uso de Muestreador

En este ejemplo, vamos a cargar un archivo CSV y aplicar una técnica de Muestreo porcentual






Importando Librerias

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## Creando la funcion de extraccion de fecha

In [2]:

def extraer_fecha(columna):
    date = []
    for d in columna:
        f = str(d).split('T')[0]
        date.append(f)
    return date

## Lectura de Archivo

In [3]:
name_file = "suburbia_boomberazo"
df = pd.read_csv(name_file+".csv",delimiter=',')
print(len(df))
df['concatenated_column'] = df['Link']+df['Autor'] + df['text']
df = df.drop_duplicates(subset = ['concatenated_column'])
df = df.drop(columns=['concatenated_column'])
print(len(df))

133329
133129


In [18]:
df.columns

Index(['Fecha', 'Hora', 'Red', 'Link', 'text_original', 'Autor', 'Likes',
       'Tipo de Mencion', 'Numero de Comentarios', 'Lenguaje', 'text',
       'Target Entities', 'clean_text', 'GEA_sentiment_str', 'Perfil',
       'Perfil_palabras', 'Motivadores de consumo',
       'Motivadores de consumo_palabras', 'Intereses', 'Intereses_palabras'],
      dtype='object')

In [4]:

df['Fecha'] = extraer_fecha(df['Fecha'])
df['Fecha'] = pd.to_datetime(df['Fecha'])

In [5]:
df['año'] = df['Fecha'].dt.isocalendar().year
df['semana'] = df['Fecha'].dt.isocalendar().week

In [6]:
# Parámetros de submuestreo
n_subsamples = 20  

porcentaje = 0.05
percent_total =len(df)*porcentaje
subsamples=[pd.DataFrame(columns=df.columns)]
percent_frac = percent_total
for i in range(n_subsamples):
    remaining_rows = df[~df.index.isin(pd.concat(subsamples).index)]
    percent_frac = (percent_total)/len(remaining_rows)
    if percent_frac<1:
        sample = remaining_rows.groupby(['año', 'semana'], group_keys=False).apply(lambda x: x.sample(frac=percent_frac))
        subsamples.append(sample)
    else:
        subsamples.append(remaining_rows)
subsamples = subsamples[1:]
for i, subsample in enumerate(subsamples):
    print(f"Tamaño de submuestra {i + 1}: {len(subsample)}")

Tamaño de submuestra 1: 6659
Tamaño de submuestra 2: 6657
Tamaño de submuestra 3: 6665
Tamaño de submuestra 4: 6650
Tamaño de submuestra 5: 6672
Tamaño de submuestra 6: 6641
Tamaño de submuestra 7: 6673
Tamaño de submuestra 8: 6637
Tamaño de submuestra 9: 6680
Tamaño de submuestra 10: 6633
Tamaño de submuestra 11: 6686
Tamaño de submuestra 12: 6632
Tamaño de submuestra 13: 6689
Tamaño de submuestra 14: 6634
Tamaño de submuestra 15: 6684
Tamaño de submuestra 16: 6636
Tamaño de submuestra 17: 6679
Tamaño de submuestra 18: 6640
Tamaño de submuestra 19: 6676
Tamaño de submuestra 20: 6606


Grafica Total


In [7]:
# Combinar todos los conteos en un único DataFrame para facilitar el graficado
conteo_total = df.groupby(['año', 'semana']).size()
# Convertir 'año' y 'semana' a una fecha real (inicio de cada semana)
conteo_total = df.groupby(['año', 'semana']).size()
ind = [ f"{i[0]}-W{i[1]}-1" for i in conteo_total.index]
ind = pd.to_datetime(ind, format="%G-W%V-%u")
plt.figure(figsize=(12, 6))
plt.plot(ind ,conteo_total.values, marker='o', linestyle='-', label='Data Total')
plt.xlabel('Fecha')
plt.ylabel('Cantidad de ocurrencias')
plt.title('Cantidad de ocurrencias por semana y año en cada submuestra')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [23]:
plt.figure(figsize=(12, 6))
#plt.plot(ind ,conteo_total.values, marker='o', linestyle='-', label='Data Total')
conteos_submuestras = []
# Calcular el conteo por semana y año para cada submuestra
for i, subsample in enumerate(subsamples):
    conteo = subsample.groupby(['año', 'semana']).size()
    conteo = conteo.reset_index(name='cantidad')  # Renombrar la columna de conteo
    conteo['submuestra'] = f'Submuestra {i + 1}'  # Etiquetar cada submuestra
    conteos_submuestras.append(conteo)

# Combinar todos los conteos en un único DataFrame para facilitar el graficado
conteo_total = pd.concat(conteos_submuestras)

# Convertir 'año' y 'semana' a una fecha real (inicio de cada semana)
conteo_total['fecha'] = conteo_total.apply(
    lambda row: pd.to_datetime(f"{row['año']}-W{row['semana']}-1", format="%G-W%V-%u"),
    axis=1
)
# Graficar cada submuestra usando fechas en el eje X
for nombre_submuestra, grupo in conteo_total.groupby('submuestra'):
    plt.plot(grupo['fecha'], grupo['cantidad'], marker='o', linestyle='-', label=nombre_submuestra)
plt.xlabel('Fecha')
plt.ylabel('Cantidad de ocurrencias')
plt.title('Cantidad de ocurrencias por semana y año en cada submuestra')
plt.legend()
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [8]:
for en,i in enumerate(subsamples):
    print(i['text'][:5])
    print("___"*10)
    i.to_csv(name_file+str(en+1) +'.csv')

57     La nueva colección de Priyanka Chopra para el ...
44     Shooters Tortugas Ninja 1995 Takara Japón, en ...
48     @Jortrej Buenas tardes, el día de hoy nuestra ...
561    Ayúdale a los #ReyesMagos pidiendo tu merch en...
936    RT @VogueMexico Este modelo de los 2000, diseñ...
Name: text, dtype: object
______________________________
10      Los modelos con este corte no perderán su vige...
38      Kim Kardashian se inspira en el imaginario del...
24      RT @JulianMaciasT No solamente eso. En este ví...
1116    ¿Existe otra mejor manera de recibir el 2023 c...
671     RT @monse_ivo "𝑺𝒐𝒍𝒐 𝒖𝒏𝒂 𝒗𝒊𝒅𝒂 𝒂𝒍 𝒔𝒆𝒓𝒗𝒊𝒄𝒊𝒐 𝒅𝒆 𝒍𝒐...
Name: text, dtype: object
______________________________
53      Una de las tendencias más auténticas de los 20...
49      Acaba de aseverarse, Dua Lipa tiene el look pe...
40      RT @SSC_CDMX Personal de la #SSC detuvo a un h...
1159    RT @itscarolives Lleva a tu waifu en tu corazó...
1432    RT @soymuynecia Una amistad va a salir hoy y c...
Name: text, dtype: ob